In [62]:
import requests
import pandas as pd
import math
import os
from scipy.stats import norm

pd.set_option('display.max_rows', None)

Given a ticker, we will query the
- stock price
- available exercise prices
- available times to expiration
- risk-free interest rate
- volatility

and use black scholes formula to return the option price

Choosing Options:

- Choose Ticker
> see all expiries
- Choose Expiry
> See options chain for that Expiry
- Choose option by code
- Plug it into formula

or

- Choose Ticker, OptType, Expiry and Strike
- Plug into formula

In [2]:
API_KEY = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"

MEMORY = {}

OPTIONS = {}

RENAME_MAP = {
    'CALL': {
        'strike': 'Strike', 
        'contractName': 'cName',
        'lastPrice': 'lastPrice', 
        'bid': 'cBidP', 
        'ask': 'cAskP',
        'volume': 'cVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'cDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    },
    'PUT': {
        'strike': 'Strike', 
        'contractName': 'pName',
        'lastPrice': 'lastPrice', 
        'bid': 'pBidP', 
        'ask': 'pAskP',
        'volume': 'pVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'pDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    }
}



In [69]:
def query_data(ticker):
    global OPTIONS
    global MEMORY
    global API_KEY
    global RENAME_MAP

    if ticker not in OPTIONS:
        url = f"https://eodhistoricaldata.com/api/options/{ticker}.US?api_token={API_KEY}"
        response = requests.request("GET", url)

        OPTIONS[ticker] = response.json()
        OPTIONS[ticker]['expiries'] = [i['expirationDate'] for i in OPTIONS[ticker]['data']]
        OPTIONS[ticker]['options_chain'] = {}

        for sample in OPTIONS[ticker]['data']:
            opt_chain = {opt['strike']:{} for opt in sample['options']["CALL"]} if len(sample['options']["CALL"]) > len(sample['options']["PUT"]) else {opt['strike']:{} for opt in sample['options']["PUT"]}
            opt_types = ['CALL', 'PUT']
            for opt_type in opt_types:
                for opt in sample['options'][opt_type]: 
                    opt_chain[opt['strike']].update({RENAME_MAP[opt_type][k]:v for k, v in opt.items() if k in ['strike', 'bid', 'ask', 'volume', 'delta', 'gamma', 'theta', 'vega', 'rho', 'contractName']})
                    MEMORY[opt['contractName']] = opt
            
            OPTIONS[ticker]['options_chain'][sample['expirationDate']] = pd.DataFrame(list(opt_chain.values()), columns = ["Theta", "Gamma", "Rho", "Vega", "cName", "cDelta", "cVolume", "cBidP", "cTheo", "cAskP", "Strike", "pBidP", "pTheo", "pAskP", "pVolume", "pDelta", 'pName'])
    
    return OPTIONS[ticker]['data']

class Option:
    def __init__(self, contract_name, memory={}):
        self.contract_name = contract_name
        self.ticker = ""
        self.generate_ticker()

        query_data(self.ticker)

        if memory:
            self.memory = {contract_name: memory}
        else:
            self.memory = MEMORY
        
        self.stock_price = OPTIONS[self.ticker]['lastTradePrice']

        self.opt_type = self.memory[contract_name]['type']
        self.exercise_price =  self.memory[contract_name]['strike']
        self.time_to_expiration =  self.memory[contract_name]['daysBeforeExpiration']

        self.volatility = self.get_annualised_log_returns(self.ticker)
        self.interest_rate = 0.0192 # 10 Year Treasury rate as at Feb 18 2022

        self.price = self.black_scholes_calculate()

    def generate_ticker(self):
        for i in self.contract_name:
            if i.isalpha():
                self.ticker += i
            else: break

    def get_annualised_log_returns(self, ticker):
        """Annualised Log Return (of Underlying) - Getting sigma from black scholes"""
        local_path = f'data/historical_prices/{ticker}_historical_data.csv'
        if not os.path.exists(local_path):
            url = f"https://eodhistoricaldata.com/api/eod/{ticker}.US?api_token={API_KEY}"
            response = requests.request("GET", url)
            if not response.ok:
                raise ConnectionError(f"Code: {response.code} - {response.text}")
            with open(local_path, 'w') as f:
                f.write(response.text)
        df = pd.read_csv(local_path)
        dfa = df.dropna().iloc[len(df)-252:len(df)]
        return sum([math.log(row['Close']/dfa.loc[i-1]['Close']) for i, row, in dfa.iterrows() if i != dfa.iloc[0].name])

    def black_scholes_calculate(self):
        d1 = (math.log(self.stock_price/self.exercise_price) + (self.interest_rate + (self.volatility**2)/2) * self.time_to_expiration) / (self.volatility * math.sqrt(self.time_to_expiration))
        d2 = d1 - (self.volatility * math.sqrt(self.time_to_expiration))
        call_price = (self.stock_price * norm.cdf(d1)) - (self.exercise_price * math.exp(-self.interest_rate * self.time_to_expiration) * norm.cdf(d2))
        put_price = -(self.stock_price * norm.cdf(-d1)) + (self.exercise_price * math.exp(-self.interest_rate * self.time_to_expiration) * norm.cdf(-d2))
        if self.opt_type == "CALL":
            return call_price
        return put_price

class OptionByChoice:
    def __init__(self, ticker, opt_type, strike, expiration):
        self.data = query_data(ticker)
        self.strike = strike
        self.expiration = expiration

        opts_date = self.find_opt_date()
        self.opt_data = self.find_opt(opts_date['options'][opt_type])
        self.option = Option(self.opt_data['contractName'], memory=self.opt_data)

    def find_opt_date(self):
        for opt in self.data:
            if opt["expirationDate"] == self.expiration:
                return opt
        raise KeyError(f"No option with date: {self.expiration}")

    def find_opt(self, opts_datetype):
        for opt in opts_datetype:
            if opt['strike'] == self.strike:
                return opt
        raise KeyError(f"No option with date: {self.expiration}, strike: {self.strike}")

In [72]:
config = {
    'ticker': "AAPL"
}

# or 

# config = {
#     'ticker': "AAPL",
#     'opt_type': 'CALL',
#     'strike': 170,
#     'expiration': '2022-02-25'
# }

# or

config = {
    'id': "AAPL220311P00155000"
}

class OptionsPricer:
    def __init__(self, config):
        self.config = config
        if {'opt_type', 'strike', 'expiration'}.issubset(set(config.keys())):
            self.option = OptionByChoice(**config).option
        elif 'id' in config.keys():
            self.option = Option(config['id'])
        else:
            query_data(config['ticker'])
            print(f"See below expiries, and then `OPTIONS['{config['ticker']}']['options_chain'][<expiry>]` to see options chains")
            print(OPTIONS[config['ticker']]['expiries'])

aapl_p = OptionsPricer(config)
aapl =  aapl_p.option
aapl.price

37.9236347932133

In [58]:
OPTIONS["AAPL"]["options_chain"]['2022-03-11']

,Theta,Gamma,Rho,Vega,cName,cDelta,cVolume,cBidP,cTheo,cAskP,Strike,pBidP,pTheo,pAskP,pVolume,pDelta,pName
0,-0.0086,0.0004,0,0.0048,AAPL220311C00105000,0.9921,2.0,60.90,NaN,63.85,105,0.01,NaN,0.05,5.0,-0.0040,AAPL220311P00105000
1,-0.0102,0.0005,0,0.0061,AAPL220311C00110000,0.9792,NaN,55.80,NaN,59.35,110,0.01,NaN,0.06,134.0,-0.0053,AAPL220311P00110000
2,-0.0148,0.0008,0,0.0092,AAPL220311C00115000,0.9504,10.0,51.05,NaN,53.20,115,0.03,NaN,0.11,6.0,-0.0084,AAPL220311P00115000
3,-0.0188,0.0012,0,0.0125,AAPL220311C00120000,0.9918,10.0,46.10,NaN,48.60,120,0.05,NaN,0.19,107.0,-0.0119,AAPL220311P00120000
4,-0.0210,0.0016,0,0.0153,AAPL220311C00125000,0.9443,20.0,41.35,NaN,43.10,125,0.08,NaN,0.16,81.0,-0.0151,AAPL220311P00125000
5,-0.0242,0.0023,0,0.0192,AAPL220311C00130000,0.9600,1.0,35.95,NaN,39.45,130,0.12,NaN,0.27,96.0,-0.0198,AAPL220311P00130000
6,-0.0341,0.0036,0,0.0287,AAPL220311C00135000,0.9788,40.0,31.65,NaN,33.20,135,0.20,NaN,0.29,85.0,-0.0319,AAPL220311P00135000
7,-0.0428,0.0053,0,0.0392,AAPL220311C00140000,0.9443,1.0,26.65,NaN,28.85,140,0.26,NaN,0.40,488.0,-0.0468,AAPL220311P00140000
8,-0.0544,0.0080,0,0.0544,AAPL220311C00145000,0.9105,33.0,21.80,NaN,23.50,145,0.48,NaN,0.60,350.0,-0.0708,AAPL220311P00145000
9,-0.0692,0.0122,0,0.0753,AAPL220311C00150000,0.8680,179.0,17.05,NaN,18.90,150,0.77,NaN,0.91,978.0,-0.1098,AAPL220311P00150000


In [14]:
os.path.exists(f'data/historical_prices/AAAPL_historical_data.csv')

False

In [47]:
x

0.20504861026313492

In [61]:
math.exp([])

2.718281828459045

In [64]:
norm.cdf(0)

0.5